<a href="https://colab.research.google.com/github/jiashenyue/salary-insights/blob/main/machine_learning_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Abalone Process**

Shenyue Jia | [jiashenyue.info](https://jiashenyue.info)

##**Overview**

**About the data**

The salary data for data scientist is collected and prepared by [Analytics Vidhya](https://datahack.analyticsvidhya.com/contest/practice-problem-big-mart-sales-iii/).

**Data dictionary**


Variable Name  | Description
-------------------|------------------
Item_Identifier	| Unique product ID
Item_Weight	| Weight of product
Item_Fat_Content	| Whether the product is low fat or regular
Item_Visibility	| The percentage of total display area of all products in a store allocated to the particular product
Item_Type	| The category to which the product belongs
Item_MRP	| Maximum Retail Price (list price) of the product
Outlet_Identifier	| Unique store ID
Outlet_Establishment_Year	| The year in which store was established
Outlet_Size	| The size of the store in terms of ground area covered
Outlet_Location_Type	| The type of area in which the store is located
Outlet_Type	| Whether the outlet is a grocery store or some sort of supermarket
Item_Outlet_Sales	| Sales of the product in the particular store. This is the target variable to be predicted.

##**1. Data Preparation**
###**1.1 Load Data**

In [105]:
# import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, \
OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(display='diagram')

This CSV data file does not contain the column names. So we read as no header. We will add the column names back later.

In [106]:
# load data
url = 'https://drive.google.com/file/d/11jKrfcHx1wcYdsfihvUrZTMGs-HpTzQY/view?usp=share_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


- We can use `df.shape` to obtain how many rows and colums in the data.

In [107]:
# check out basic dataframe information
df.shape
print(f'There are {df.shape[0]} rows, and {df.shape[1]} columns.')
print(f'The rows represent {df.shape[0]} observations, and the columns represent {df.shape[1]-1} features and 1 target variable.')

There are 8523 rows, and 12 columns.
The rows represent 8523 observations, and the columns represent 11 features and 1 target variable.


In [108]:
# make a copy of the original data to avoid data manipulation
sales = df.copy()

###**1.2 Check for Duplicated, Missing, or Erroneous Data**

####**1.2.1 Check missing values**

In [109]:
# Display the sum of missing values
sales.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

####**1.2.2 Check duplicated rows**

In [110]:
# Check to see if there are any duplicate rows
duplicateRows = sales[df.duplicated()]
print(f'There are {len(duplicateRows)} duplicated rows in the dataframe')

There are 0 duplicated rows in the dataframe


- No duplicated rows detected

####**1.3 Check inconsistencies**

In [111]:
# Look at the info from the data
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [112]:
# check value counts for each object column
for col in sales.columns:
  print(f"The unique values of the feature {col} are:\n")
  print(df[col].value_counts())
  print("\n")

The unique values of the feature Item_Identifier are:

FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64


The unique values of the feature Item_Weight are:

12.150    86
17.600    82
13.650    77
11.800    76
15.100    68
          ..
7.275      2
7.685      1
9.420      1
6.520      1
5.400      1
Name: Item_Weight, Length: 415, dtype: int64


The unique values of the feature Item_Fat_Content are:

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64


The unique values of the feature Item_Visibility are:

0.000000    526
0.076975      3
0.162462      2
0.076841      2
0.073562      2
           ... 
0.013957      1
0.110460      1
0.124646      1
0.054142      1
0.044878      1
Name: Item_Visibility, Length: 7880, dtype: int64


The unique values of the feature Item_Type are:

Fruits and Ve

Here are some discrepancies we found in `Item_Fat_Content` column:
- `reg` should be `Regular`
- `low fat` should be `Low Fat`
- `LF` should be `Low Fat`

In [113]:
sales.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [114]:
# replace the inconsistent categories
sales.replace(to_replace=['LF', 'low fat'], value='Low Fat', inplace=True)
sales.replace('reg','Regular', inplace=True)

In [115]:
# show what the column looks like after correction
sales['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

##**2. Splitting data**
###**2.1 Identify X and y variables**

**X and y variables**
- Target variable (y)
  - `Item_Outlet_Sales`
- Features (X), all the rest variables
  - `Item_Identifier`
  - `Item_Weight`
  - `Item_Fat_Content`
  - `Item_Visibility`
  - `Item_Type`
  - `Item_MRP`
  - `Outlet_Identifier`
  - `Outlet_Establishment_Year`
  - `Outlet_Size`
  - `Outlet_Location_Type`
  - `Outlet_Type`

In [116]:
# check if there are missing values in the Item_Outlet_Sales
sales['Item_Outlet_Sales'].isna().sum()

0

- There is no missing value in the target variable `Item_Outlet_Sales`

###**2.2 Split the data into training/testing parts**

- `Item_Outlet_Sales` column is our target variable.

In [117]:
# split X and y, you are predicting price
X = sales.drop(columns=['Item_Outlet_Sales']).copy()
y = sales['Item_Outlet_Sales'].copy()

# split training and test
# set random_state to 42 for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30 , random_state=42)

##**3. Create transformers for data**

We will use 
- StandardScaler
- OneHotEncoder
- OrdinalEncoder and Imputers

###**3.1 Instantiate the transformers**

- We have missing values in the following columns
  - `type`
  - `calories per serving`
  - `grams of fat`
  - `grams of sugars`
  - `calories per serving`
- Only `type` is an `object` column, the rest are all numeric columns
- We can consider using the most frequent value to fill the missing values in `type`.
- For the numeric values, it is safer to use **median** as we cannot guarantee that each numeric column follows the normal distribution and the mean value can represent the middle status of the data.
- These strategies will be applied when we create a imputator

In [118]:
#instantiate the StandardScaler, OneHotEncoder, OrdinalEncoder and Imputers
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

median_imputer = SimpleImputer(strategy='median')
freq_imputer = SimpleImputer(strategy='most_frequent')

#this is showing for learning purpose but we are not using it here. 
missing_imputer = SimpleImputer(strategy='constant', fill_value='missing')

###**3.2 Build an ordinal encoder**

In [119]:
#create a list of ordinal labels, in order of least to most

outlet_size_labels = ['Small','Medium','High']
outlet_location_labels = ['Tier 3','Tier 2','Tier 1']
fat_content_labels = ['Low Fat','Regular Fat']

#combine the ordered list in the order that the columns appear
ordered_labels = [outlet_size_labels, outlet_location_labels, fat_content_labels]

#handle_unknown is 'error' by default.  That's a good place to start
#but it may cause problems in a production model.  
ordinal = OrdinalEncoder(categories = ordered_labels)

###**3.2 Create a Pipeline Numeric Data and a Pipeline for Categorical Data**

- The **nominal** variables are
  - `Item_Identifier`
  - `Item_Fat_Content`
  - `Item_Type`
  - `Outlet_Identifier`
  - `Outlet_Type`
- The **ordinal** variables are
  - `Item_Fat_Content`
  - `Outlet_Size`
  - `Outlet_Location_Type`
- The **numeric** variables are
  - `Item_Weight`
  - `Item_Visibility`
  - `Item_MRP`
  - `Outlet_Establishment_Year`

We may consider dropping the `Item_Identifier` and `Outlet_Identifier` columns as they contain information that is not useful for our model.

In [120]:
# drop two identifier columns
sales = sales.drop(['Item_Identifier','Outlet_Identifier'], axis=1)
sales.dtypes

Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [121]:
# Setup the pipelines for the numeric and categorical data

num_pipeline = make_pipeline(median_imputer, scaler)
ord_pipeline = make_pipeline(freq_imputer, ordinal)
nom_pipeline = make_pipeline(missing_imputer, ohe)

###**3.3 Create Tuples to Pair Pipelines with Columns**

In [122]:
# Create column lists for objects and a number selector
ordinal_cols = ['Outlet_Size','Outlet_Location_Type']
nominal_cols = ['Item_Fat_Content','Item_Identifier','Item_Fat_Content','Item_Type',
                'Outlet_Identifier','Outlet_Type']

num_selector = make_column_selector(dtype_include='number')

# Setup the tuples to pair the processors with the make column selectors
numeric_tuple = (num_pipeline, num_selector)
ordinal_tuple = (ord_pipeline, ordinal_cols)
nominal_tuple = (nom_pipeline, nominal_cols)

###**3.4 Create Column Transformer to Apply Different Preprocessing to Different Columns**
- Assemble the pipelines into a `preprocesser`
- We only have numeric and nominal variables in our data
- We use `drop` for the remainder

In [123]:
# Instantiate the make column transformer
preprocessor = make_column_transformer(ordinal_tuple, 
                                          numeric_tuple, 
                                          nominal_tuple, 
                                          remainder='drop')

- Display the `ColumnTransformer`

**PACE MARKER**

**There are some errors to be fixed to finish this part.**

In [124]:
# Fit the column transformer on the X_train
preprocessor.fit(X_train)

ValueError: ignored

##**4. Transform our data and display the result**

In [ ]:
# Transform the X_train and the X_test
X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [ ]:
# show the shape of X_train after transformation
X_train_transformed.shape

(49, 12)

- Display the data after transformation
- Remember, here `X_train_transformed` is an Array, not a dataframe anymore

In [ ]:
# display X_train first five rows
X_train_transformed[:5]

array([[ 0.88757452,  0.02359436,  2.06275064,  1.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [-1.00334511, -1.59497854, -1.60735761,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ],
       [-1.00334511,  0.02359436, -0.35952081,  1.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [-0.05788529,  0.48604376, -0.35952081,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 3.72395398,  0.25481906, -0.35952081,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ]])

In [ ]:
# check if there are any NaN values after transformation
np.isnan(X_train_transformed).sum()

0